# Mini Project - Vehicle Collisions


resources:
[Vehicle Collision Data](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95)

[Socrata API Doc](https://dev.socrata.com/foundry/data.cityofnewyork.us/h9gi-nx95)

[zipcode to neighborhood](https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm)



## Part 2 - Perform the Analysis (Data Transformation + Visualization)


### Environment Setup

In [0]:
!pip install sodapy

In [0]:
import pandas as pd
import altair as alt
from sodapy import Socrata
import datetime as dt

In [0]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [0]:
client = Socrata("data.cityofnewyork.us", "FSftCRf0oX5vPMmtWkvtr6DXD")
results = client.get("h9gi-nx95", limit=2000000)

In [0]:
data_df = pd.DataFrame.from_records(results)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1676892 entries, 0 to 1676891
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   crash_date                     1676892 non-null  object
 1   crash_time                     1676892 non-null  object
 2   cross_street_name              239705 non-null   object
 3   number_of_persons_injured      1676875 non-null  object
 4   number_of_persons_killed       1676861 non-null  object
 5   number_of_pedestrians_injured  1676892 non-null  object
 6   number_of_pedestrians_killed   1676892 non-null  object
 7   number_of_cyclist_injured      1676892 non-null  object
 8   number_of_cyclist_killed       1676892 non-null  object
 9   number_of_motorist_injured     1676892 non-null  object
 10  number_of_motorist_killed      1676892 non-null  object
 11  contributing_factor_vehicle_1  1672360 non-null  object
 12  contributing_factor_vehicle_

### Clean Data
Turn all text strings to uppercase to optimize grouping

In [0]:
text_columns = ['on_street_name', 'off_street_name', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'vehicle_type_code1', 'vehicle_type_code2', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5' ]
for column in text_columns:
  data_df[column] = data_df[column].str.upper() 


Clean data: fill all empty cells/NaN with 'MISSING' so it can be differentiated from other reasons such as 'UNSPECIFIED' 

In [0]:
data_df[text_columns] = data_df[text_columns].fillna('MISSING')

Extract datetime, month, day, weekday, weeknumber, hour

In [0]:
data_df["crash_dt"] = pd.to_datetime(data_df['crash_date'])
data_df["year"] = data_df.crash_dt.dt.year
data_df["month"] = data_df.crash_dt.dt.month
data_df["weeknumber"] = data_df.crash_dt.dt.weekofyear
data_df["day"] = data_df.crash_dt.dt.day
data_df["dayofyear"] = data_df.crash_dt.dt.dayofyear
data_df["weekday"] = data_df.crash_dt.dt.weekday
data_df['hour'] = pd.to_datetime(data_df['crash_time'], format='%H:%M').dt.hour

### Top Contributing Factors and Vehicle Types

In [0]:
source = data_df.groupby('contributing_factor_vehicle_1').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('contributing_factor_vehicle_1:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
    #color='borough'
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

In [0]:
source = data_df.groupby('contributing_factor_vehicle_2').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('contributing_factor_vehicle_2:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

In [0]:
source = data_df.groupby('vehicle_type_code1').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('vehicle_type_code1:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

In [0]:
source = data_df.groupby('vehicle_type_code2').size().sort_values(ascending=False).to_frame('counts').reset_index().nlargest(15, 'counts')

bars = alt.Chart().mark_bar().encode(
    x=alt.X('vehicle_type_code2:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q'),
)

texts = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='counts:Q'
)

alt.layer(bars, texts, data=source).properties(
    width=800,
    height=400
)

alt.LayerChart(...)

### Year/Day Heatmap

In [0]:
source = data_df.groupby(['year', 'dayofyear']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('dayofyear:O', title='Day of Year', axis=alt.Axis(labelAngle=-0, tickCount=30)),
    alt.Y('year:O', title='Year', axis=alt.Axis(labelAngle=-0)),
    alt.Color('counts', bin=alt.Bin(extent=[min, max], step=100), 
        scale=alt.Scale(scheme='yellowgreenblue'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=800
)


alt.Chart(...)

### Month/Hour Heatmap

In [0]:
source = data_df.groupby(['month', 'hour']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('hour:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('month:O'),
    alt.Color('counts', bin=alt.Bin(extent=[min, max], step=1000), 
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=600,
    title='Month By Hour '
)


alt.Chart(...)

### Weekday/Hour Heatmap

In [0]:
# source=source.loc[data_df["zip"]]
for index, row in data_df.iterrows():
    # print("hello")
    if row["hour"]>=8 and  row["hour"]<10:
      data_df.at[index, "peak"]="Morning Peak"
    elif row["hour"]>=16 and  row["hour"]<18:
      data_df.at[index, "peak"]="Evening Peak"
    else:
      data_df.at[index, "peak"]="Not Peak"
# print(data_df.head())
source = data_df.groupby(['weekday', 'peak']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('peak:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('weekday:O'),
    alt.Color('counts', bin=alt.Bin(extent=[min, max], step=2000), 
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=600,
    title='Weekday By Hour '
)



alt.Chart(...)

### Zipcode

In [0]:
source = data_df.groupby(['borough', 'zip_code']).size().reset_index(name='counts')
source=source.set_index("borough")

bronx=source.loc["BRONX"]
bronx=bronx.reset_index()
bronx=bronx.loc[bronx["counts"]!=0]
bronx=bronx.loc[bronx["counts"]>=bronx["counts"].mean()]

brooklyn=source.loc["BROOKLYN"]
brooklyn=brooklyn.reset_index()
brooklyn=brooklyn.loc[brooklyn["counts"]!=0]
brooklyn=brooklyn.loc[brooklyn["counts"]>=brooklyn["counts"].mean()]

manhattan=source.loc["MANHATTAN"]
manhattan=manhattan.reset_index()
manhattan=manhattan.loc[manhattan["counts"]!=0]
manhattan=manhattan.loc[manhattan["counts"]>=manhattan["counts"].mean()]

queens=source.loc["QUEENS"]
queens=queens.reset_index()
queens=queens.loc[queens["counts"]!=0]
queens=queens.loc[queens["counts"]>=queens["counts"].mean()]

staten_island=source.loc["STATEN ISLAND"]
staten_island=staten_island.reset_index()
staten_island=staten_island.loc[staten_island["counts"]!=0]
# staten_island=staten_island.loc[staten_island["counts"]>=staten_island["counts"].mean()]

chart1 = alt.Chart(bronx).mark_bar().encode(
    alt.X('counts:Q'),
    alt.Y('zip_code:O',sort='-x'),
    
).properties(
    height=200,
    width=200,
     title='Bronx-Collision over Zipcode'
)


chart2 = alt.Chart(brooklyn).mark_bar().encode(
    alt.X('counts:Q'),
    alt.Y('zip_code:O',sort='-x'),
    
).properties(
    height=200,
    width=200,
    title='Brooklyn-Collision over Zipcode'
)

chart3 = alt.Chart(manhattan).mark_bar().encode(
    alt.X('counts:Q'),
    alt.Y('zip_code:O',sort='-x'),
    
).properties(
    height=200,
    width=200,
    title='Manhattan-Collision over Zipcode'
)


chart4 = alt.Chart(queens).mark_bar().encode(
    alt.X('counts:Q'),
    alt.Y('zip_code:O',sort='-x'),
    
).properties(
  height=200,
    width=200,
    title='Queens-Collision over Zipcode'
)

chart5 = alt.Chart(staten_island).mark_bar().encode(
    alt.X('counts:Q'),
    alt.Y('zip_code:O',sort='-x'),
    
).properties(
    height=200,
    width=200,
    title='Staten Island-Collision over Zipcode'
)

chart1 | chart2 | chart3 | chart4 | chart5
# bronx.head()
# chart1

In [0]:
source = data_df.groupby(['borough', 'zip_code']).size().reset_index(name='counts')
source['percentage'] = source.counts / source.counts.sum()
source = source.sort_values('percentage', ascending=False).groupby('borough').head(5)

bars = alt.Chart().mark_bar().encode(
    x=alt.X('zip_code:N', sort='-y', axis=alt.Axis(labelAngle=-90)),
    y=alt.Y('percentage:Q', axis=alt.Axis(format='%', labelAngle=-0), title="Percentage of Total Collisions"),
    color='borough',
    text='counts:Q'
)

alt.layer(bars, data=source).properties(
    height=300,
    width=600,
    title='Top 5 most dangerous zipcodes of each borough, sorted'
)



In [0]:
source = data_df.groupby(['borough', 'zip_code', 'peak']).size().reset_index(name='counts')
source=source.loc[source["borough"]=="BROOKLYN"]
source=source.loc[source["peak"]=="Morning Peak"]
source
source = source.sort_values('counts', ascending=False).groupby([ 'peak']).head()
base = alt.Chart(source).mark_bar().encode(
    x=alt.X('zip_code:O', title='Zip Code',axis=alt.Axis(labelAngle=-0),sort="-y"),
    y=alt.Y('counts:Q', title="Total Collisions"),
    # color='peak:N',
).properties(
    height=400,
    width=400,
    title='Most dangerous zipcode in each borough over the years'
).facet(
    column="peak"
)

source = data_df.groupby(['borough', 'zip_code', 'peak']).size().reset_index(name='counts')
source=source.loc[source["borough"]=="BROOKLYN"]
source=source.loc[source["peak"]=="Evening Peak"]
source
source = source.sort_values('counts', ascending=False).groupby([ 'peak']).head()
base1 = alt.Chart(source).mark_bar().encode(
    x=alt.X('zip_code:O', title='Zip Code',axis=alt.Axis(labelAngle=-0),sort="-y"),
    y=alt.Y('counts:Q', title="Total Collisions"),
    # color='peak:N',
).properties(
    height=400,
    width=400,
    title='Most dangerous zipcode in each borough over the years'
).facet(
    column="peak"
)
base | base1

# texts = base.mark_text(
#     align='center',
#     baseline='bottom',
# ).encode(
#     text='zip_code:Q'
# )
# alt.layer(base, texts, data=source)

alt.HConcatChart(...)

In [0]:
source = data_df.groupby(['borough', 'zip_code', 'year']).size().reset_index(name='counts')
source = source.sort_values('counts', ascending=False).groupby(['borough', 'year']).head(1)
base = alt.Chart(source).mark_area().encode(
    x=alt.X('year:O', title='Year', axis=alt.Axis(labelAngle=-0)),
    y=alt.Y('counts:Q', title="Total Collisions"),
    color='zip_code:N',
).properties(
    height=200,
    width=300,
)

texts = base.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='zip_code:Q'
)

alt.layer(base, texts, data=source).facet(
    column='borough',
    title='Most dangerous zipcode in each borough over the years'
)

### Borough

In [0]:
data_df['borough'] = data_df['borough'].astype('str') 
bor = data_df.groupby(['borough']).size().to_frame('counts')

data_df=data_df.loc[data_df['borough']!='nan']
data_df['borough'].unique()
total=data_df['borough'].count()
print(total)
for index, row in bor.iterrows():
    bor.at[index, "normalized"]=float((row["counts"]/total)*100)
bor=bor.reset_index()
bor.head()

chart1 = alt.Chart(bor).mark_bar().encode(
    alt.X('counts:Q'),
    alt.Y('borough:O',sort='-x'),
    color='borough:N'
).properties(
    height=200,
    width=200,
    title='Collision over Borough-Count '
)

chart2 = alt.Chart(bor).mark_bar().encode(
    alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
    alt.Y('borough:O',sort='-x'),
    color='borough:N'
).properties(
    height=200,
    width=200,
    title='Collision over Borough-% '
)

chart1 | chart2

In [0]:

bor = data_df.groupby(['borough',"year"]).size().to_frame('counts')

data_df=data_df.loc[data_df['borough']!='nan']
data_df['borough'].unique()
total=data_df['year'].count()
bor["normalized"]=0.0
for index, row in bor.iterrows():
    bor.at[index, "normalized"]=float((row["counts"]/total)*100)
bor=bor.reset_index()
bor.head()


# chart2 = alt.Chart(bor).mark_bar().encode(
#     alt.X('counts:Q',axis=alt.Axis(title='Count')),
#     alt.Y('year:O',sort='-y'),
    
# ).properties(
#     height=200,
#     width=200,
#     title='Collision over Borough countover years'
# ).facet(
#     column='borough:N',
#     title='Type of vehicles in collision by Borough-% '
# )

alt.Chart(bor).mark_line().encode(
    alt.X('year:O'),
    alt.Y('normalized:Q',axis=alt.Axis(title='counts')),
    color='borough:N'
).properties(
    width=450,
    height=300,
    title='Collision over Borough-count over year'
)

In [0]:
def normalize(df,data_df,col):
  bor = data_df.groupby(['borough']).size().to_frame('counts')
  bronx=pd.to_numeric(bor.loc["BRONX"].counts)
  brooklyn=pd.to_numeric(bor.loc["BROOKLYN"].counts)
  manhattan=pd.to_numeric(bor.loc["MANHATTAN"].counts)
  queens=pd.to_numeric(bor.loc["QUEENS"].counts)
  staten_island=pd.to_numeric(bor.loc["STATEN ISLAND"].counts)
  df["totalcount"]=0

  # for index, row in df.iterrows():
  #   if row["borough"]=="BRONX":
  #     df.at[index, "totalcount"]=bronx
  #   elif row["borough"]=="BROOKLYN":
  #     df.at[index, "totalcount"]=brooklyn
  #   elif row["borough"]=="MANHATTAN":
  #     df.at[index, "totalcount"]=manhattan
  #   elif row["borough"]=="QUEENS":
  #     df.at[index, "totalcount"]=queens
  #   elif row["borough"]=="STATEN ISLAND":
  #     df.at[index, "totalcount"]=staten_island
  # df=df.loc[df["totalcount"]!=0]
  # # print("hi")
  totalcount=data_df[col].count()
  df["normalized"]=0.0
  for index, row in df.iterrows():
    # print("hello")
    df.at[index, "normalized"]=float(float(row["counts"])/float(totalcount))*100
  print(df.head())

  return df

source = data_df.groupby(['weekday', 'borough']).size().to_frame('counts').reset_index()

source=normalize(source,data_df,"weekday")
print(source.columns)
source.head()

In [0]:
color= ['red', 'steelblue', 'chartreuse', '#F4D03F', '#D35400', '#7D3C98']
alt.Chart(source).mark_line().encode(
    alt.X('weekday:O'),
    alt.Y('normalized:Q',axis=alt.Axis(title='percentage')),
    color='borough:N'
).properties(
    width=450,
    height=300*2,
    title='Collision over Borough-% over weekday'
)

In [0]:
source = data_df.groupby(['hour','borough']).size().to_frame('counts').reset_index()
source=normalize(source,data_df,"hour")
# source=source.set_index("weekday")
# source.dtypes
# source=source.loc[[1,2,3,4,5]]
source=source.reset_index()
alt.Chart(source).mark_line().encode(
    alt.X('hour:O'),
    alt.Y('normalized:Q',axis=alt.Axis(title='percentage')),
    color='borough:N'
).properties(
    width=400,
    height=300*2,
    title='Collision in Boroughs-% by hour'
)

for index, row in data_df.iterrows():
    # print("hello")
    if row["hour"]>=8 and  row["hour"]<10:
      data_df.at[index, "peak"]="Morning Peak"
    elif row["hour"]>=16 and  row["hour"]<18:
      data_df.at[index, "peak"]="Evening Peak"
    else:
      data_df.at[index, "peak"]="Not Peak"
print(data_df.head())

In [0]:
source = data_df.groupby(['contributing_factor_vehicle_1','borough',"peak"]).size().to_frame('counts').reset_index()
source=source.loc[source["contributing_factor_vehicle_1"]!="UNSPECIFIED"]
source=source.loc[source["contributing_factor_vehicle_1"]!="1"]
source=source.loc[source["contributing_factor_vehicle_1"]!="80"]
source=source.loc[source["peak"]!="Not Peak"]
source["contributing_factor_vehicle_1"].unique()

source=normalize(source,data_df,"contributing_factor_vehicle_1")
# source=source.loc[source["normalized"]>=source["normalized"].mean()]
source=source.loc[source["normalized"]!=0]
# source["normalized"].mean()
# source=source.set_index("weekday")
# source.dtypes
# source=source.loc[[1,2,3,4,5]]
source=source.reset_index()
alt.Chart(source).mark_bar().encode(
    alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
    alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
    color='borough:N'
).properties(
    width=200,
    height=200,
).facet(
    column='borough:N',
    title='Factors of collision by Borough-% '
)


In [0]:
source = data_df.groupby(['contributing_factor_vehicle_1','borough',"peak"]).size().to_frame('counts').reset_index()
source=source.loc[source["contributing_factor_vehicle_1"]!="UNSPECIFIED"]
source=source.loc[source["contributing_factor_vehicle_1"]!="1"]
source=source.loc[source["contributing_factor_vehicle_1"]!="80"]
source=source.loc[source["contributing_factor_vehicle_1"]!="MISSING"]
source=source.loc[source["peak"]!="Not Peak"]
# source["contributing_factor_vehicle_1"].unique()

source=normalize(source,data_df,"peak")
source=source.loc[source["normalized"]>=source["normalized"].mean()]
source=source.loc[source["normalized"]!=0]
# source=source.loc[source["borough"]=="STATEN ISLAND"]
source=source.reset_index()
# alt.Chart(source).mark_bar().encode(
#     alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
#     alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
#     color='borough:N'
# ).properties(
#     width=400,
#     height=600,
#     title='Factors of collision by  Borough-% '
# )
# alt.Chart(source).mark_bar().encode(
#     alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
#     alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
#     color='borough:N'
# ).properties(
#     width=200,
#     height=200,
# ).facet(
#     column='borough:N',
#     title='Factors of collision by Borough-% '
# )


heatmap=alt.Chart(source).mark_bar().encode(
    alt.X('normalized:Q'),
    alt.Y('contributing_factor_vehicle_1:O',sort='-x'),
).properties(
    width=400,
    height=600,
    title='Contributing factor vehicle 1 in NYC by peak hour'
).facet(
    column='peak:N',
    title='Contributing factor vehicle 1 in NYC by peak hour '
)
 

In [0]:
heatmap

In [0]:

source = data_df.loc[data_df["zip_code"]=="11207"]
source = source.groupby(['contributing_factor_vehicle_1','borough',"peak"]).size().to_frame('counts').reset_index()
source=source.loc[source["contributing_factor_vehicle_1"]!="UNSPECIFIED"]
source=source.loc[source["contributing_factor_vehicle_1"]!="1"]
source=source.loc[source["contributing_factor_vehicle_1"]!="80"]
source=source.loc[source["contributing_factor_vehicle_1"]!="MISSING"]
source=source.loc[source["peak"]!="Not Peak"]

# source["contributing_factor_vehicle_1"].unique()

source=normalize(source,data_df,"peak")
source=source.loc[source["normalized"]>=source["normalized"].mean()]
source=source.loc[source["normalized"]!=0]
# source=source.loc[source["borough"]=="STATEN ISLAND"]
source=source.reset_index()
# alt.Chart(source).mark_bar().encode(
#     alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
#     alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
#     color='borough:N'
# ).properties(
#     width=400,
#     height=600,
#     title='Factors of collision by  Borough-% '
# )
# alt.Chart(source).mark_bar().encode(
#     alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
#     alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
#     color='borough:N'
# ).properties(
#     width=200,
#     height=200,
# ).facet(
#     column='borough:N',
#     title='Factors of collision by Borough-% '
# )


heatmap=alt.Chart(source).mark_bar().encode(
    alt.X('normalized:Q'),
    alt.Y('contributing_factor_vehicle_1:O',sort='-x'),
).properties(
    width=400,
    height=600,
    title='Contributing factor vehicle 1 in 11207 by peak hour'
).facet(
    column='peak:O',
    title='Contributing factor vehicle 1 in 11207 by peak hour '
)
heatmap

In [0]:
source = data_df.groupby(['contributing_factor_vehicle_1','borough',"hour"]).size().to_frame('counts').reset_index()
source=source.loc[source["contributing_factor_vehicle_1"]!="UNSPECIFIED"]
source=source.loc[source["contributing_factor_vehicle_1"]!="1"]
source=source.loc[source["contributing_factor_vehicle_1"]!="80"]
source=source.loc[source["contributing_factor_vehicle_1"]!="MISSING"]

# source["contributing_factor_vehicle_1"].unique()

source=normalize(source,data_df,"hour")
source=source.loc[source["normalized"]>=source["normalized"].mean()]
source=source.loc[source["normalized"]!=0]
source=source.loc[source["borough"]=="BROOKLYN"]
source=source.reset_index()
# alt.Chart(source).mark_bar().encode(
#     alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
#     alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
#     color='borough:N'
# ).properties(
#     width=400,
#     height=600,
#     title='Factors of collision by  Borough-% '
# )


heatmap=alt.Chart(source).mark_rect().encode(
    alt.X('hour:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('contributing_factor_vehicle_1:O'),
    alt.Color('normalized')
).properties(
    width=400,
    height=600,
    title='Contributing factor vehicle 1 in BROOKLYN by hour'
)

heatmap 

In [0]:
source = data_df.groupby(['contributing_factor_vehicle_2','borough',"hour"]).size().to_frame('counts').reset_index()
source=source.loc[source["contributing_factor_vehicle_2"]!="UNSPECIFIED"]
source=source.loc[source["contributing_factor_vehicle_2"]!="1"]
source=source.loc[source["contributing_factor_vehicle_2"]!="80"]
source=source.loc[source["contributing_factor_vehicle_2"]!="MISSING"]

# source["contributing_factor_vehicle_1"].unique()

source=normalize(source,data_df,"hour")
source=source.loc[source["normalized"]>=source["normalized"].mean()]
source=source.loc[source["normalized"]!=0]
source=source.loc[source["borough"]=="BROOKLYN"]
source=source.reset_index()
# alt.Chart(source).mark_bar().encode(
#     alt.Y('contributing_factor_vehicle_1:O',sort='-x',axis=alt.Axis(title='Factor')),
#     alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
#     color='borough:N'
# ).properties(
#     width=400,
#     height=600,
#     title='Factors of collision by  Borough-% '
# )


alt.Chart(source).mark_rect().encode(
    alt.X('hour:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('contributing_factor_vehicle_2:O'),
    alt.Color('normalized')
).properties(
    width=400,
    height=600,
    title='Contributing factor vehicle2 in BROOKLYN by hour'
)

In [0]:
# source = data_df.loc[data_df["borough"]=="BROOKLYN"]
# sourcedf = source.groupby(['zip_code','peak']).size().to_frame('counts').reset_index()
# sourcedf.head()
# alt.Chart(sourcedf).mark_circle().encode(
#     x = alt.X('peak:N'),
#     y = alt.Y('zip_code:N'),
#     size= 'counts:Q'
# ).properties(
#     width=600,
#     height=600,
#     title='Brooklyn zipcode collision by hour'
# )

source = data_df.loc[data_df["zip_code"]=="11207"]
source = source.groupby(['weekday', 'hour']).size().to_frame('counts').reset_index()

max = source['counts'].max()
min = source['counts'].min()

alt.Chart(source).mark_rect().encode(
    alt.X('hour:O', axis=alt.Axis(labelAngle=-0)),
    alt.Y('weekday:O'),
    alt.Color('counts')
    
).properties(
    width=600,
    title='Weekday By Hour '
)
# source.head()

In [0]:
source = data_df.loc[data_df["borough"]=="BROOKLYN"]
source = source.loc[source["zip_code"]=="11207"]

sourcedf = source.groupby(['cross_street_name','peak']).size().to_frame('sum').reset_index()
sourcedf.head()


sourcem=sourcedf.loc[sourcedf["peak"]=="Morning Peak"]
sourcem=sourcem.loc[sourcem["sum"]!=0]
sourcem=sourcem.loc[sourcem["sum"]>2]
# alt.Chart(sourcedf).mark_rect().encode(
#     alt.X('peak:N'),
#     alt.Y('cross_street_name:N'),
#     alt.Color('counts')
# ).properties(
#     width=400,
#     height=500,
#     title='BROOKLYN dangerous cross streets'
# )
morning= alt.Chart(sourcem).mark_bar().encode(
    alt.Y('cross_street_name:O',sort='-x'),
    alt.X('sum:Q'),
).properties(
    width=200,
    height=200,
    title='Cross streets at 11207 morning peak time'
)

sourcedf = source.groupby(['cross_street_name','peak']).size().to_frame('sum').reset_index()
sourcee=sourcedf.loc[sourcedf["peak"]=="Evening Peak"]
sourcee=sourcee.loc[sourcee["sum"]!=0]
sourcee=sourcee.loc[sourcee["sum"]>2]
eve= alt.Chart(sourcee).mark_bar().encode(
    alt.Y('cross_street_name:O',sort='-x',axis=alt.Axis(title='Factor')),
    alt.X('sum:Q'),
).properties(
    width=200,
    height=200,
    title='Cross streets at 11207 evening peak time'
)


sourcen=sourcedf.loc[sourcedf["peak"]=="Not Peak"]
sourcen=sourcen.loc[sourcen["sum"]!=0]
sourcen=sourcen.loc[sourcen["sum"]>7]
notp= alt.Chart(sourcen).mark_bar().encode(
    alt.Y('cross_street_name:O',sort='-x',axis=alt.Axis(title='Factor')),
    alt.X('sum:Q'),
).properties(
    width=200,
    height=200,
    title='Cross streets at 11207 non-peak time'
)


morning |eve |notp
# | eve | notp

In [0]:
source = data_df.loc[data_df["borough"]=="BROOKLYN"]
source = source.loc[source["zip_code"]=="11207"]

sourcedf = source.groupby(['cross_street_name','peak']).size().to_frame('counts').reset_index()
sourcee=sourcedf.loc[sourcedf["peak"]=="Evening Peak"]
sourcee=sourcee.loc[sourcee["counts"]!=0]
sourcee=sourcee.loc[sourcee["counts"]>2]
alt.Chart(sourcee).mark_bar().encode(
    alt.Y('cross_street_name:O',sort='-x',axis=alt.Axis(title='Factor')),
    alt.X('counts:Q',axis=alt.Axis(title='percentage')),
).properties(
    width=200,
    height=200,
)

In [0]:
#maps of streets
source = data_df.loc[data_df["borough"]=="BROOKLYN"]
source = source.loc[source["zip_code"]=="11207"]

In [0]:
source = data_df.groupby(['vehicle_type_code1','borough']).size().to_frame('counts').reset_index()

source=normalize(source,data_df)
# source=source.reset_index()
source=source.loc[source["normalized"]!=0]
source=source.loc[source["normalized"]>=source["normalized"].mean()]
alt.Chart(source).mark_bar().encode(
    alt.Y('vehicle_type_code1:O',sort="-x",axis=alt.Axis(title='Vehicle Type')),
    alt.X('normalized:Q',axis=alt.Axis(title='percentage')),
    color='borough:N'
).properties(
    width=200,
    height=200,
).facet(
    column='borough:N',
    title='Type of vehicles in collision by Borough-% '
)

### Passenger Group

In [0]:
#  number_of_persons_injured      1672483 non-null  object
#  4   number_of_persons_killed       1672469 non-null  object
#  5   number_of_pedestrians_injured  1672500 non-null  object
#  6   number_of_pedestrians_killed   1672500 non-null  object
#  7   number_of_cyclist_injured      1672500 non-null  object
#  8   number_of_cyclist_killed       1672500 non-null  object
#  9   number_of_motorist_injured     1672500 non-null  object
#  10  number_of_motorist_killed      1672500 non-null  object
# groups = data_df.groupby(['borough']).size().to_frame('counts')
alldata=data_df.groupby(["year"]).size().to_frame('counts').reset_index()


nohurt=data_df.loc[(data_df["number_of_persons_killed"]=="0") & (data_df["number_of_persons_injured"]=="0")]
nohurt_gruop=nohurt.groupby(["year","borough"]).size().to_frame('counts').reset_index()
nohurt_gruop=nohurt_gruop.rename(columns={"counts":"noinjury"})
result = pd.merge(nohurt_gruop, alldata, on=["year"])
for index, row in result.iterrows():
    # print("hello")
    result.at[index, "normalized"]=float(float(row["noinjury"])/float(row["counts"]))*100

result=result.drop(["noinjury","counts"],axis=1)
result=result.rename(columns={"normalized":"noinjury"})


injured=data_df.loc[data_df["number_of_persons_injured"]!="0"]
injured_gruop=injured.groupby(["year","borough"]).size().to_frame('sum').reset_index()
injured_gruop=injured_gruop.rename(columns={"sum":"injured"})
result1 = pd.merge(injured_gruop, alldata, on=["year"])
for index, row in result1.iterrows():
    # print("hello")
    result1.at[index, "normalized"]=float(float(row["injured"])/float(row["counts"]))*100
result1=result1.drop(["injured","counts"],axis=1)
result1=result1.rename(columns={"normalized":"injured"})


killed=data_df.loc[data_df["number_of_persons_killed"]!="0"]
killed_group=killed.groupby(["year","borough"]).size().to_frame('sum').reset_index()
killed_group=killed_group.rename(columns={"sum":"killed"})
result2 = pd.merge(killed_group, alldata, on=["year"])
for index, row in result2.iterrows():
    # print("hello")
    result2.at[index, "normalized"]=float(float(row["killed"])/float(row["counts"]))*100
result2=result2.drop(["killed","counts"],axis=1)
result2=result2.rename(columns={"normalized":"killed"})



final = pd.merge(result, result1, on=["year","borough"])

final = pd.merge(final, result2, on=["year","borough"])
final.head()

chart1 = alt.Chart(final).mark_bar().encode(
    alt.X('noinjury:Q',stack="normalize"),
    alt.Y('year:O'),
    color='borough:N'
).properties(
    height=200,
    width=200,
    title='Persons without Injury by  Borough and year'
)

chart2 = alt.Chart(final).mark_bar().encode(
    alt.X('injured:Q',stack="normalize"),
    alt.Y('year:O'),
    color='borough:N'
).properties(
    height=200,
    width=200,
     title='Persons with Injury by  Borough and year'
)

chart3 = alt.Chart(final).mark_bar().encode(
    alt.X('killed:Q',stack="normalize"),
    alt.Y('year:O'),
    color='borough:N'
).properties(
    height=200,
    width=200,
     title='Persons Killed by  Borough and year'
)

# chart1 | chart2 | chart3
alt.vconcat(chart1, chart2,chart3)

In [0]:
data_df[["number_of_pedestrians_injured","number_of_pedestrians_killed","number_of_cyclist_injured","number_of_cyclist_killed","number_of_motorist_injured","number_of_motorist_killed"]] = data_df[["number_of_pedestrians_injured","number_of_pedestrians_killed","number_of_cyclist_injured","number_of_cyclist_killed","number_of_motorist_injured","number_of_motorist_killed"]].apply(pd.to_numeric) 

# alldata.head()

In [0]:
source = data_df.loc[data_df["borough"]=="BROOKLYN"]
source = source.loc[source["zip_code"]=="11207"]
groups=source[["year","borough","number_of_pedestrians_injured","number_of_pedestrians_killed","number_of_cyclist_injured","number_of_cyclist_killed","number_of_motorist_injured","number_of_motorist_killed"]]

groups=groups.reset_index()
for index, row in groups.iterrows():
    groups.at[index, "Cyclist"]=int(row["number_of_cyclist_injured"])+int(row["number_of_cyclist_killed"])
    groups.at[index, "Pedastrian"]=int(row["number_of_pedestrians_injured"])+int(row["number_of_pedestrians_killed"])
    groups.at[index, "Motorist"]=int(row["number_of_motorist_injured"])+int(row["number_of_motorist_killed"])

In [0]:
groups.head()

In [0]:
groups_year=groups.groupby(["year","borough"]).sum()
groups_year=groups_year.reset_index()
groups_year=groups_year[["year","borough","Cyclist","Pedastrian","Motorist"]]
groups_year.head()

groups_year=groups_year.melt(id_vars=["year","borough"], 
        var_name="group", 
        value_name="Value")
groups_year.head()
groups_year=groups_year.groupby(["year","group","borough"]).sum()
groups_year.head()
groups_year=groups_year.reset_index()


In [0]:

# alt.Chart(groups_year).mark_rect().encode(
#     alt.X('Value:Q',axis=alt.Axis(title='Percentage')),
#     alt.Y('year:O'),
#     color="borough:N",
#     column='group:N'
# ).properties(
#     height=200,
#     width=200,
#     title='Group of passengers Borough and year'
# )

# alt.Chart(sourcedf).mark_rect().encode(
#     alt.X('hour:N'),
#     alt.Y('cross_street_name:N'),
#     alt.Color('counts')
# ).properties(
#     width=400,
#     height=1500,
#     title='BROOKLYN dangerous cross streets'
# )


# alt.Chart(groups_year).mark_rect().encode(
#     alt.X('year:O', axis=alt.Axis(labelAngle=-0)),
#     alt.Y('borough:O'),
#     alt.Color('Value')
# ).properties(
#     width=400,
#     height=600,
# ).facet(
#     column='group:N',
#     title='Group of passengers Borough and year '
# )
alt.Chart(groups_year).mark_line().encode(
    alt.X('year:O'),
    alt.Y('Value:Q',axis=alt.Axis(title='percentage')),
    color='group:N',
).properties(
    width=400,
    height=300,
    title='Group of passengers Brooklyn-11207 and year'
).facet(
    column='borough:N',
    title='Factors of collision by Borough-% '
)


In [0]:
source_map = data_df.groupby(['borough', 'zip_code']).size().reset_index(name='counts')

source_map=source_map.set_index("borough")
brooklyn=source_map.loc["BROOKLYN"]
rooklyn=brooklyn.reset_index()
brooklyn=brooklyn.loc[brooklyn["counts"]!=0]
brooklyn
#brooklyn=brooklyn.loc[brooklyn["counts"]>=brooklyn["counts"].mean()]

#brooklyn_data = data_df.loc[data_df["borough"]=="BROOKLYN"]
#quantile_doamin = [brooklyn["counts"].min(), brooklyn['counts'].quantile(.25), brooklyn['counts'].median(), brooklyn['counts'].quantile(.75), brooklyn['counts'].max()]
to_plot = brooklyn.groupby('zip_code').size().reset_index(name='count')
to_plot

borough_geo = alt.Data(url='boroughs.geojson.txt', format=alt.DataFormat(property='features',type='json'))

nyc_background = alt.Chart(borough_geo).mark_geoshape(
    stroke='white',
    strokeWidth=2
).encode(
    color=alt.value('#eee'),
).properties(
    width=500,
    height=300
)

nyc_background
# create geo data for brooklyn 
alt.Chart(borough_geo).mark_geoshape().encode(
    color='count:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(to_plot, 'zip_code', ['counts'])
).properties(
    width=500,
    height=300
)